# ModelVerfication.jl Python Interface

Here we report how it is possible to use ModelVerification directly in Python:

## Installation dependencies for ModelVerfication.jl Python interface:

  
To have compatibility with our toolbox, we created an interface to merge Julia and Python code. Hence, to run the toolbox using a python script follow these steps:

1) Download and Install Julia from here: https://julialang.org/downloads/ and select the stable version.

2) Copy and paste this line in the terminal: 

> export PATH="$PATH:where-is-located-the-folder/julia-1.9.3/bin"

Note: Replace 'julia-1.8.5-linux-x86_64' with your Julia's version that was previously downloaded.

1) Start Julia by running `julia` in the terminal, if everything is correctly installed, the Julia prompt should appear.

2) Set the `ENV["PYTHON"]` variable in Julia:


> ENV["PYTHON"] = "/Users/your-name/location-of-you-python-bin-file"

Note: replace the path with your python path. The last path part should be the same.

5) Now install PyCall and other packages using Pkg, Julia’s package manager:

> using Pkg

> Pkg.add("PyCall")
 
6) pip install julia


## Using the toolbox

In [1]:
from julia.api import Julia
jl = Julia(compiled_modules=False)
from julia import Main

In [2]:
# first let's import some useful packages in Julia (it will require at least 5 min)
Main.eval("using Revise; using ModelVerification;using LazySets")

Here you can convert your .h5, .pt(h) model file into onnx using one of the scripts provided in the 'converters/' folder... let's now first test a forward propagion and then define the safety property to be verified!

In [4]:
# First, let's load the model
onnx_path = "models/small_nnet.onnx"
Main.eval(f'toy_model = ModelVerification.build_flux_model("{onnx_path}")')


<PyCall.jlwrap Chain(Dense(1 => 2, relu), Dense(2 => 2, relu), Dense(2 => 1, relu))>

In [5]:
print("Test propagation onnx:")
output = Main.eval("toy_model([0.60014254])")
print(output)

Test propagation onnx:
[68.90342096]


As reported in the tutorial suppose we want to verify the following safety property.  We consider the following DNN called "small_nnet":

<p align="center">
<img src="images_tutorial/toyDNN.png" alt="Drawing" align="center" style="width: 600px;"/>
</p>

The model has a single input node, two hidden layers composed of two ReLU nodes, and a single output node. Hence this DNN is a non-linear function that maps $\mathbb{R}^1 \to \mathbb{R}^1$. Suppose we want to verify if all the possible inputs in a vector that assume values between $\mathcal{X}=[-2.5, 2.5]$ are mapped in the vector $\mathcal{Y}=[18.5, 114.5]$. 

In [7]:
property = {"X": [[-2.5, 2.5]], "Y":[18.5, 114.5]}
lower = [l[0] for l in property["X"]]
upper = [l[1] for l in property["X"]]

print("The lower bound is: ", lower)
print("The upper bound is: ", upper)

Main.eval(f'X = Hyperrectangle(low={lower}, high={upper})')
Main.eval(f'Y = Hyperrectangle(low=[{property["Y"][0]}], high=[{property["Y"][1]}])')

# let's instantiate the Problem for the ModelVerification.jl toolbox
Main.eval("problem = Problem(toy_model, X, Y)")

# we instantiate the 3 main components for the verification process
Main.eval("search_method = BFS(max_iter=100, batch_size=1)")
Main.eval("split_method = Bisect(1)")
Main.eval("solver = Ai2()")

The lower bound is:  [-2.5]
The upper bound is:  [2.5]


<PyCall.jlwrap Ai2z()>

In [9]:
# run the verification part
# we then start the verification process using the verify function of ModelVerification.jl
result = Main.eval("verify(search_method, split_method, solver, problem)")

# the verification result is stored in 'result'. Let's print it!
print("The result is: ", result)

# the verification result is stored in 'result.status'. Let's see if the property holds...
print("The property", result.status, "!")

The result is:  <PyCall.jlwrap BasicResult(:holds)>
The property holds !
